In [182]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

In [183]:
df = pd.read_csv("train.csv")

In [149]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


'Loan_ID' - unique 
'Gender' - more males than females, maybe just include null as an option since non binary is a thing 
'Married' - null values are included, could just put no 
'Dependents' - 0, more than half have zero 
'Education' - 
'Self_Employed' - 32 missing values, could impute no because there is typically a lower chance and majority is not self employed 
'ApplicantIncome' - huge standard deviation because there is a disproportionality in peoples incomes 
'CoapplicantIncome'-huge standard deviation
'LoanAmount' - mean value of the loan amounts that have similar loan amount term as the na people (360 I beleive because 19 of 22 have this but this is very common in general too) 
'Loan_Amount_Term' - utilize mean because there is a low std and the only other similarities in data is 0 dependents and not self employed.. but this is most of the data
'Credit_History' - has the highest number of missing values, maybe put 0.. it is highly correlated with the loan status 
'Property_Area' - look into the loan status for each of the property areas 
'Loan_Status' - much more loan status yes than no 

Possibly Skewed Data: 
- loan status
- gender 
- dependents 
- self employment 

Imputation Tasks: 
-smote on possibly skewed data points 
-married - no 
-gender- include another value null or just choose female
-self employed- no 
-loan amount- mean value 
-dependents - 0 (or mean)
-loan amount term- mean 


Changing the data types to contain booleans where applicable 

In [150]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [188]:
# cols = ['Married','Self_Employed','Loan_Status','Dependents']
# changed_type = ['bool','bool','bool','string']
# for i in range(4):
#     df[cols[i]]=df[cols[i]].astype(changed_type[i])
to_replace = {'Married':{'Yes':True,'No':False},'Self_Employed':{'Yes':True,'No':False},'Loan_Status':{'Y':True,'N':False},'Credit_History':{1:True,0:False},'Education':{'Graduate':True,'Not Graduate':False}}
df.replace(to_replace=to_replace,inplace=True)
df.astype({'Married':'bool','Self_Employed':'bool','Loan_Status':'bool','Dependents':'object','Education':'bool'}).dtypes

Loan_ID               object
Gender                object
Married                 bool
Dependents            object
Education               bool
Self_Employed           bool
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History          bool
Property_Area         object
Loan_Status             bool
dtype: object

Description of Features: 
Gender- add other value 
Married- true if married, false if not 
Dependents- numerical value has no statistical meaning so we take it as an object. Default is zero because likely people skip if they do not have kids
Education- True if graduate, false otherwise 
Self_Employed- True if self employed, false otherwise 
ApplicantIncome- it is the income of the applicant.. utilize the median value to fill the na values 
Coapplicant Income- there are many that are blank and are converted to zeros.. so we can consider adding a new feature that asks if there is a coapplicant 
LoanAmount- how much loan they want.. take the median value for those nan
LoanAmountTerm- take the median value for those nan 
CreditHistory- is either 1 if good and 0 if bad.. we convert the dtype to boolean.. so true if good and false if bad 
Property Area- which area are they buying, or what type of area do they live in.. object 
Loan Status- True if approved, false otherwise

Fill in the missing values with the appropriate values 

In [189]:
values = {'Married':False,'Gender':'Other','Self_Employed':False,'LoanAmount':df['LoanAmount'].median(),'Dependents':'0','Loan_Amount_Term':df['Loan_Amount_Term'].median(),'Credit_History':False}
df.fillna(value=values,inplace=True)

In [190]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,False,0,True,False,5849,0.0,128.0,360.0,True,Urban,True
1,LP001003,Male,True,1,True,False,4583,1508.0,128.0,360.0,True,Rural,False
2,LP001005,Male,True,0,True,True,3000,0.0,66.0,360.0,True,Urban,True
3,LP001006,Male,True,0,False,False,2583,2358.0,120.0,360.0,True,Urban,True
4,LP001008,Male,False,0,True,False,6000,0.0,141.0,360.0,True,Urban,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,False,0,True,False,2900,0.0,71.0,360.0,True,Rural,True
610,LP002979,Male,True,3+,True,False,4106,0.0,40.0,180.0,True,Rural,True
611,LP002983,Male,True,1,True,False,8072,240.0,253.0,360.0,True,Urban,True
612,LP002984,Male,True,2,True,False,7583,0.0,187.0,360.0,True,Urban,True


In [174]:
df.isna().sum()


Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [46]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


Which features have the highest missing values 

In [110]:
cols = df.columns
vals = [(col,len(df[df[col].isna()])) for col in cols]
vals.sort(key=lambda x:x[1], reverse=True)

for val in vals:
    print(val,'\n')


('Credit_History', 50) 

('Self_Employed', 32) 

('LoanAmount', 22) 

('Dependents', 15) 

('Loan_Amount_Term', 14) 

('Gender', 13) 

('Married', 3) 

('Loan_ID', 0) 

('Education', 0) 

('ApplicantIncome', 0) 

('CoapplicantIncome', 0) 

('Property_Area', 0) 

('Loan_Status', 0) 



Credit History

In [127]:
df['Credit_History'].unique() #some number from 0-1.somthing or nan
#df[df['Credit_History'].dtype != 'float64']
df_only_na = df[df['Credit_History'].isna()]
print(df_only_na['ApplicantIncome']) #compare this income and possibly coapplicant income 
#to that of others to see if it is lower or if there is another trend 

16      3596
24      3717
30      4166
42      2400
79      3333
83      6000
86      3333
95      6782
117     2214
125     3692
129     6080
130    20166
156     6000
181     1916
187     2383
198     3416
219     4283
236     5746
237     3463
259     4931
260     6083
279     4100
309     7667
313     5746
317     2058
318     3541
323     3166
348     6333
363     3013
377     4310
392     2583
395     3276
411     6256
444     7333
449     2769
451     1958
460     2083
473     2500
490     2699
491     5333
497     4625
503     4050
506    20833
530     1025
533    11250
544     3017
556     2667
565     4467
583     1880
600      416
Name: ApplicantIncome, dtype: int64


Self Employed

In [44]:
df['Self_Employed'].unique
len(df[df['Self_Employed']=='No']) #500
len(df[df['Self_Employed']=='Yes']) #82
#probably not self employed, maybe not employed idk adhu pathi data illa 

82

Loan Amount

In [125]:
u_values = df['LoanAmount'].unique()
# len(u_values) #203
# print(u_values)
mask = np.isin(u_values, 'nan')
values = u_values[u_values.astype(int) == u_values]
# len(values) #203
# print(df[df['LoanAmount'].isin(values)])
# df[df['LoanAmount'].dtype != int]
# LA = df.dropna(subset='LoanAmount')
#print(LA) #maybe put in a mean value 
# df_nan = df[df['LoanAmount'].isna()]
# print(df_nan.head())
# print(df_nan['Loan_Amount_Term']) #the term tends to be 
unique_loan_term = df['Loan_Amount_Term'].unique()
count = [(val,len(df_nan[df_nan['Loan_Amount_Term']==val])) for val in unique_loan_term]
count.sort(key=lambda x:x[1], reverse=True)
for val in count:
    print(val)


(360.0, 19)
(180.0, 2)
(240.0, 1)
(120.0, 0)
(nan, 0)
(60.0, 0)
(300.0, 0)
(480.0, 0)
(36.0, 0)
(84.0, 0)
(12.0, 0)


c:\Users\jeyas\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Dependents

In [109]:
len(df[df['Dependents'].isna()]) #15
len(df[df['Dependents']=='0']) #345
#more than half have zero 
len(df[df['Dependents']=='1']) #102

15

Loan Amount Term 

In [126]:
#loan amount term 
df[df['Loan_Amount_Term'].isna()] #mostly not self employed and 0 dependents, std for term mean is low so we could use the mean 

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
19,LP001041,Male,Yes,0,Graduate,NaN,2600,3500.0,115.0,NaN,1.0,Urban,Y
36,LP001109,Male,Yes,0,Graduate,No,1828,1330.0,100.0,NaN,0.0,Urban,N
44,LP001136,Male,Yes,0,Not Graduate,Yes,4695,0.0,96.0,NaN,1.0,Urban,Y
45,LP001137,Female,No,0,Graduate,No,3410,0.0,88.0,NaN,1.0,Urban,Y
73,LP001250,Male,Yes,3+,Not Graduate,No,4755,0.0,95.0,NaN,0.0,Semiurban,N
112,LP001391,Male,Yes,0,Not Graduate,No,3572,4114.0,152.0,NaN,0.0,Rural,N
165,LP001574,Male,Yes,0,Graduate,No,3707,3166.0,182.0,NaN,1.0,Rural,Y
197,LP001669,Female,No,0,Not Graduate,No,1907,2365.0,120.0,NaN,1.0,Urban,Y
223,LP001749,Male,Yes,0,Graduate,No,7578,1010.0,175.0,NaN,1.0,Semiurban,Y
232,LP001770,Male,No,0,Not Graduate,No,3189,2598.0,120.0,NaN,1.0,Rural,Y


Education 

In [136]:
print(len(df[df['Education']=="Graduate"])) #480 
print(len(df[df['Education']=='Not Graduate'])) #134 
#see any differences in income and loan status with graduates and non graduates 

480
134


Property Area

In [131]:
PAs = df['Property_Area'].unique()
areas = [(pa, len(df[df['Property_Area']==pa])) for pa in PAs]
for area in areas:
    print(area)
#look into the loan status for each of the property area types 

('Urban', 202)
('Rural', 179)
('Semiurban', 233)


Loan Status 

In [134]:
print(len(df[df['Loan_Status'] == 'Y'])) #422
print(len(df[df['Loan_Status']=='N'])) #192 


422
192


In [47]:
#total number of missing values 
df.isna().size

7982

In [48]:
#probably no duplicates 
df.drop_duplicates()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [49]:
profile = ProfileReport(df, title="Profiling Report")

In [50]:
profile.to_file("profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Exporting the df to a csv.. will be imported later to do feature cleaning

In [191]:
df.to_csv(path_or_buf='/Users/jeyas/Downloads/project_poonchers/dc_df.csv')